In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
# Load dataset
data = pd.read_csv('/content/car.data', header=None)
data.columns = ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_Boot', 'Safety', 'Class']  # Assign column names

In [3]:
# Encode categorical features using LabelEncoder
label_encoders = {}
for col in data.columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])  # Encode labels numerically

# Split the dataset into features and target
X = data.drop('Class', axis=1).values  # Features
y = data['Class'].values  # Target

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalize features to have mean 0 and variance 1

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Create custom Dataset class for PyTorch
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [6]:
# Define the MLP model class
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPModel, self).__init__()
        layers = []

        for i, hidden_size in enumerate(hidden_layers):
            layers.append(nn.Linear(input_size if i == 0 else hidden_layers[i-1], hidden_size))
            if activation_fn == 'relu':
                layers.append(nn.ReLU())
            elif activation_fn == 'sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_fn == 'tanh':
                layers.append(nn.Tanh())
            elif activation_fn == 'softmax':
                layers.append(nn.Softmax(dim=1))

        layers.append(nn.Linear(hidden_layers[-1], len(set(y))))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [7]:
# Define function to train and evaluate the model
def train_and_evaluate(hidden_layers, activation_fn, learning_rate, batch_size, epochs):
    # Initialize dataset and dataloaders
    train_dataset = CustomDataset(X_train, y_train)
    test_dataset = CustomDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, activation_fn=activation_fn).to(device)
    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")  # Verbose output

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    return accuracy

In [8]:
# Experiment 1: Comparing Hidden Layers and Neurons
hidden_layers_options = [[4], [8], [16], [32], [64], [128]]
results_hidden_layers = []
for hidden_layers in hidden_layers_options:
    accuracy = train_and_evaluate(hidden_layers, activation_fn='relu', learning_rate=0.01, batch_size=32, epochs=50)
    results_hidden_layers.append({'hidden_layers': hidden_layers, 'accuracy': accuracy})


Epoch [1/50], Loss: 1.0203
Epoch [2/50], Loss: 0.7019
Epoch [3/50], Loss: 0.6646
Epoch [4/50], Loss: 0.6241
Epoch [5/50], Loss: 0.6135
Epoch [6/50], Loss: 0.5853
Epoch [7/50], Loss: 0.5894
Epoch [8/50], Loss: 0.5773
Epoch [9/50], Loss: 0.5559
Epoch [10/50], Loss: 0.5568
Epoch [11/50], Loss: 0.5450
Epoch [12/50], Loss: 0.5167
Epoch [13/50], Loss: 0.5007
Epoch [14/50], Loss: 0.5019
Epoch [15/50], Loss: 0.5089
Epoch [16/50], Loss: 0.4880
Epoch [17/50], Loss: 0.4832
Epoch [18/50], Loss: 0.4872
Epoch [19/50], Loss: 0.4866
Epoch [20/50], Loss: 0.4706
Epoch [21/50], Loss: 0.4673
Epoch [22/50], Loss: 0.4708
Epoch [23/50], Loss: 0.4547
Epoch [24/50], Loss: 0.4605
Epoch [25/50], Loss: 0.4407
Epoch [26/50], Loss: 0.4417
Epoch [27/50], Loss: 0.4373
Epoch [28/50], Loss: 0.4288
Epoch [29/50], Loss: 0.4242
Epoch [30/50], Loss: 0.4223
Epoch [31/50], Loss: 0.4158
Epoch [32/50], Loss: 0.4117
Epoch [33/50], Loss: 0.4125
Epoch [34/50], Loss: 0.4127
Epoch [35/50], Loss: 0.4105
Epoch [36/50], Loss: 0.4124
E

In [9]:
# Experiment 2: Comparing Activation Functions
activation_functions = ['relu', 'sigmoid', 'tanh', 'softmax']
results_activation_functions = []
for activation_fn in activation_functions:
    accuracy = train_and_evaluate(hidden_layers=[32], activation_fn=activation_fn, learning_rate=0.01, batch_size=32, epochs=50)
    results_activation_functions.append({'activation_function': activation_fn, 'accuracy': accuracy})

Epoch [1/50], Loss: 0.8493
Epoch [2/50], Loss: 0.6064
Epoch [3/50], Loss: 0.4875
Epoch [4/50], Loss: 0.3719
Epoch [5/50], Loss: 0.3093
Epoch [6/50], Loss: 0.2529
Epoch [7/50], Loss: 0.2179
Epoch [8/50], Loss: 0.1921
Epoch [9/50], Loss: 0.1689
Epoch [10/50], Loss: 0.1556
Epoch [11/50], Loss: 0.1399
Epoch [12/50], Loss: 0.1327
Epoch [13/50], Loss: 0.1178
Epoch [14/50], Loss: 0.1138
Epoch [15/50], Loss: 0.1080
Epoch [16/50], Loss: 0.1044
Epoch [17/50], Loss: 0.0962
Epoch [18/50], Loss: 0.0900
Epoch [19/50], Loss: 0.0802
Epoch [20/50], Loss: 0.0797
Epoch [21/50], Loss: 0.0782
Epoch [22/50], Loss: 0.0692
Epoch [23/50], Loss: 0.0655
Epoch [24/50], Loss: 0.0633
Epoch [25/50], Loss: 0.0620
Epoch [26/50], Loss: 0.0632
Epoch [27/50], Loss: 0.0579
Epoch [28/50], Loss: 0.0571
Epoch [29/50], Loss: 0.0537
Epoch [30/50], Loss: 0.0544
Epoch [31/50], Loss: 0.0545
Epoch [32/50], Loss: 0.0475
Epoch [33/50], Loss: 0.0469
Epoch [34/50], Loss: 0.0518
Epoch [35/50], Loss: 0.0441
Epoch [36/50], Loss: 0.0411
E

In [10]:
# Experiment 3: Comparing Epochs
epochs_options = [1, 10, 25, 50, 100, 250]
results_epochs = []
for epochs in epochs_options:
    accuracy = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=0.01, batch_size=32, epochs=epochs)
    results_epochs.append({'epochs': epochs, 'accuracy': accuracy})


Epoch [1/1], Loss: 0.7964
Epoch [1/10], Loss: 0.7931
Epoch [2/10], Loss: 0.5845
Epoch [3/10], Loss: 0.4588
Epoch [4/10], Loss: 0.3577
Epoch [5/10], Loss: 0.3025
Epoch [6/10], Loss: 0.2544
Epoch [7/10], Loss: 0.2244
Epoch [8/10], Loss: 0.1870
Epoch [9/10], Loss: 0.1668
Epoch [10/10], Loss: 0.1489
Epoch [1/25], Loss: 0.8102
Epoch [2/25], Loss: 0.6044
Epoch [3/25], Loss: 0.4855
Epoch [4/25], Loss: 0.3771
Epoch [5/25], Loss: 0.3018
Epoch [6/25], Loss: 0.2682
Epoch [7/25], Loss: 0.2189
Epoch [8/25], Loss: 0.1975
Epoch [9/25], Loss: 0.1720
Epoch [10/25], Loss: 0.1516
Epoch [11/25], Loss: 0.1421
Epoch [12/25], Loss: 0.1430
Epoch [13/25], Loss: 0.1228
Epoch [14/25], Loss: 0.1123
Epoch [15/25], Loss: 0.1065
Epoch [16/25], Loss: 0.1056
Epoch [17/25], Loss: 0.0933
Epoch [18/25], Loss: 0.0958
Epoch [19/25], Loss: 0.0908
Epoch [20/25], Loss: 0.0813
Epoch [21/25], Loss: 0.0858
Epoch [22/25], Loss: 0.0921
Epoch [23/25], Loss: 0.0741
Epoch [24/25], Loss: 0.0689
Epoch [25/25], Loss: 0.0649
Epoch [1/50]

In [11]:
# Experiment 4: Comparing Learning Rates
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
results_learning_rates = []
for learning_rate in learning_rates:
    accuracy = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=learning_rate, batch_size=32, epochs=50)
    results_learning_rates.append({'learning_rate': learning_rate, 'accuracy': accuracy})

Epoch [1/50], Loss: 339.3041
Epoch [2/50], Loss: 5.7993
Epoch [3/50], Loss: 1.1252
Epoch [4/50], Loss: 0.9100
Epoch [5/50], Loss: 1.0542
Epoch [6/50], Loss: 0.8951
Epoch [7/50], Loss: 0.9757
Epoch [8/50], Loss: 1.0851
Epoch [9/50], Loss: 1.1045
Epoch [10/50], Loss: 0.8065
Epoch [11/50], Loss: 0.9237
Epoch [12/50], Loss: 1.1141
Epoch [13/50], Loss: 1.5351
Epoch [14/50], Loss: 1.7963
Epoch [15/50], Loss: 1.2554
Epoch [16/50], Loss: 0.9578
Epoch [17/50], Loss: 1.1620
Epoch [18/50], Loss: 1.0803
Epoch [19/50], Loss: 1.3108
Epoch [20/50], Loss: 0.8163
Epoch [21/50], Loss: 1.0439
Epoch [22/50], Loss: 0.9306
Epoch [23/50], Loss: 0.8911
Epoch [24/50], Loss: 0.9575
Epoch [25/50], Loss: 0.9491
Epoch [26/50], Loss: 0.9807
Epoch [27/50], Loss: 1.4206
Epoch [28/50], Loss: 0.9755
Epoch [29/50], Loss: 0.9081
Epoch [30/50], Loss: 1.5156
Epoch [31/50], Loss: 0.8772
Epoch [32/50], Loss: 0.8990
Epoch [33/50], Loss: 1.4534
Epoch [34/50], Loss: 1.0000
Epoch [35/50], Loss: 0.9886
Epoch [36/50], Loss: 0.8735

In [12]:
# Experiment 5: Comparing Batch Sizes
batch_sizes = [16, 32, 64, 128, 256, 512]
results_batch_sizes = []
for batch_size in batch_sizes:
    accuracy = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=0.01, batch_size=batch_size, epochs=50)
    results_batch_sizes.append({'batch_size': batch_size, 'accuracy': accuracy})

Epoch [1/50], Loss: 0.7082
Epoch [2/50], Loss: 0.4663
Epoch [3/50], Loss: 0.3303
Epoch [4/50], Loss: 0.2510
Epoch [5/50], Loss: 0.2112
Epoch [6/50], Loss: 0.1717
Epoch [7/50], Loss: 0.1532
Epoch [8/50], Loss: 0.1395
Epoch [9/50], Loss: 0.1175
Epoch [10/50], Loss: 0.1121
Epoch [11/50], Loss: 0.0963
Epoch [12/50], Loss: 0.0894
Epoch [13/50], Loss: 0.0820
Epoch [14/50], Loss: 0.0752
Epoch [15/50], Loss: 0.0691
Epoch [16/50], Loss: 0.0659
Epoch [17/50], Loss: 0.0655
Epoch [18/50], Loss: 0.0582
Epoch [19/50], Loss: 0.0613
Epoch [20/50], Loss: 0.0492
Epoch [21/50], Loss: 0.0497
Epoch [22/50], Loss: 0.0446
Epoch [23/50], Loss: 0.0516
Epoch [24/50], Loss: 0.0460
Epoch [25/50], Loss: 0.0395
Epoch [26/50], Loss: 0.0459
Epoch [27/50], Loss: 0.0428
Epoch [28/50], Loss: 0.0499
Epoch [29/50], Loss: 0.0347
Epoch [30/50], Loss: 0.0398
Epoch [31/50], Loss: 0.0347
Epoch [32/50], Loss: 0.0296
Epoch [33/50], Loss: 0.0245
Epoch [34/50], Loss: 0.0282
Epoch [35/50], Loss: 0.0233
Epoch [36/50], Loss: 0.0245
E

In [19]:
# Consolidate and save results
hidden_layers_df = pd.DataFrame(results_hidden_layers)
activation_functions_df = pd.DataFrame(results_activation_functions)
epochs_df = pd.DataFrame(results_epochs)
learning_rates_df = pd.DataFrame(results_learning_rates)
batch_sizes_df = pd.DataFrame(results_batch_sizes)

# Save each result separately
hidden_layers_df.to_csv('mlp_hidden_layers_results.csv', index=False)
activation_functions_df.to_csv('mlp_activation_functions_results.csv', index=False)
epochs_df.to_csv('mlp_epochs_results.csv', index=False)
learning_rates_df.to_csv('mlp_learning_rates_results.csv', index=False)
batch_sizes_df.to_csv('mlp_batch_sizes_results.csv', index=False)

print("Results saved for each experiment.")

Results saved for each experiment.
